In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.4-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cruise-lr').getOrCreate()

In [3]:
data = spark.read.csv('data/cruise_ship_info.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
data.count()

158

In [5]:
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [6]:
for item in data.head():
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [7]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [8]:
#Convert from String to Int
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_Index')

In [9]:
data = indexer.fit(data).transform(data)
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_Index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [10]:
#Convert columns to features
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_Index']

In [12]:
assembler = VectorAssembler(inputCols=['Age',
                                         'Tonnage',
                                         'passengers',
                                         'length',
                                         'cabins',
                                         'passenger_density',
                                         'Cruise_line_Index'],
                           outputCol="features")

In [13]:
output = assembler.transform(data)

In [14]:
output.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_Index=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [15]:
final_data = output.select(['features', 'crew'])
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [16]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               117|
|   mean| 8.040598290598302|
| stddev|3.3876187574547294|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [18]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                41|
|   mean| 7.090975609756096|
| stddev|3.7699249361744904|
|    min|               0.6|
|    max|              19.1|
+-------+------------------+



In [19]:
from pyspark.ml.regression import LinearRegression

In [20]:
lr = LinearRegression(labelCol='crew')

In [21]:
lr_model = lr.fit(train_data)

In [22]:
test_result = lr_model.evaluate(test_data)

In [23]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.03691576655213247|
| -1.1889018189510407|
| -1.1889018189510407|
| -1.4857615504101886|
| 0.49009132432717806|
| -0.5091939030033821|
| -0.3810822841184631|
|-0.33914411987774695|
|-0.36274627404573323|
|-0.07347279593776435|
| -0.5555951700139552|
|   7.170376421477192|
|-0.31844265627511725|
|-0.20198750193721127|
|  -0.541876591802378|
| -1.4145818298256563|
| -0.4222928461182178|
|  0.5370351015827275|
|  0.1374738706762617|
|-0.00942100592452...|
+--------------------+
only showing top 20 rows



In [24]:
test_result.rootMeanSquaredError

1.2634656826761042

In [25]:
test_result.r2

0.8848708335467861

In [26]:
unlabeled_data = test_data.select('features')

In [27]:
predictions = lr_model.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,133.5,39.59,...|13.093084233447868|
|[6.0,30.276999999...| 4.738901818951041|
|[6.0,30.276999999...| 4.738901818951041|
|[6.0,90.0,20.0,9....|10.485761550410189|
|[6.0,113.0,37.82,...|11.509908675672822|
|[7.0,116.0,31.0,9...|12.509193903003382|
|[8.0,110.0,29.74,...|11.981082284118463|
|[9.0,59.058,17.0,...| 7.739144119877747|
|[9.0,110.0,29.74,...|11.962746274045733|
|[9.0,116.0,26.0,9...|11.073472795937764|
|[10.0,86.0,21.14,...| 9.755595170013954|
|[11.0,110.0,29.74...| 11.92962357852281|
|[12.0,2.329,0.94,...|0.9184426562751172|
|[12.0,25.0,3.88,5...|3.0719875019372114|
|[13.0,61.0,13.8,7...| 6.541876591802378|
|[13.0,63.0,14.4,7...| 6.724581829825656|
|[13.0,76.0,18.74,...| 8.922292846118218|
|[13.0,101.509,27....|10.962964898417273|
|[14.0,33.0,4.9,5....|3.1025261293237385|
|[14.0,83.0,17.5,9...| 9.459421005924527|
+--------------------+------------

In [30]:
from pyspark.sql.functions import corr
data.corr('crew', 'passengers')

0.9152341306065384

In [32]:
data.corr('crew', 'cabins')

0.9508226063578497